## Data Quality checks
- Check fo missing data or gaps in time series 
- Check Outliers (if applicable)
- Schema validation - already done during ingestion phase

In [1]:
import os
import json
from dotenv import load_dotenv
from datetime import datetime, timedelta, timezone
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, min, max, expr, to_date, count, mean, stddev, col

In [2]:
load_dotenv()
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home"
# Get Jar path needed for spark session
# For simplicity using locally downloaded jars for delta format
cwd = os.getcwd()
if cwd.endswith("notebooks"):
    proj_dir = os.path.abspath("..")
else:
    proj_dir = cwd
jar_dir = os.path.join(proj_dir, "jars")
jar1 = os.path.join(jar_dir, "delta-spark_2.13-4.0.0.jar")
jar2 = os.path.join(jar_dir, "delta-storage-4.0.0.jar")

In [3]:
spark = SparkSession.builder.appName("EnergyUseCase") \
            .config("spark.jars", f"{jar1},{jar2}") \
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
            .getOrCreate()

25/06/29 19:22:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Checking DQ of public power data ..

In [4]:
power_data = spark.read.format("delta").load(f"{proj_dir}/data/silver/public_power_data")  

In [5]:
# DQ functions 
def check_for_gaps():
    """ Records per day should have 96 distinct values because of 15 mins interval"""

    res = power_data.withColumn("day", to_date("timestamp")) \
        .groupBy("production_type","day") \
        .agg(count("*").alias("records_per_day")) \
        .orderBy("production_type", "day") 
    cnt = res.select("records_per_day").distinct().count()
    invalid_days = res.filter(col("records_per_day") != 96).select("day").distinct().collect()
    invalid_days_dict = [row.asDict() for row in invalid_days]
    return {"check": f"Data Gaps", "status": "Pass" if cnt == 1 else "Fail", "failed_records": f"failed on days {invalid_days_dict}"}

def check_missing_field(id_column):
    missing_id_count = power_data.filter(col(id_column).isNull()).count()
    return {"check": f"{id_column} not null", "status": "Pass" if missing_id_count == 0 else "Fail", "failed_records": missing_id_count}


def get_outliers():
    
    stats = power_data.groupBy("production_type").agg(
        mean("net_power_produced").alias("mean"),
        stddev("net_power_produced").alias("std")
    )
    df_with_stats = power_data.join(stats, on="production_type", how="left")
    outliers = df_with_stats.filter(
        (col("net_power_produced") > col("mean") + 3 * col("std")) |
        (col("net_power_produced") < col("mean") - 3 * col("std"))
    )
    summary= outliers.groupBy("production_type").count()
    outlier_info = {row['production_type']: row['count'] for row in summary.collect()}
    return {"check": f"Data Outliers", "status": "Pass" if bool(outlier_info) == False else "Fail", "failed_records": f"Have outliers {outlier_info}"}
    
    

In [6]:
def run_data_quality_checks():
    results = []

    results.append(check_missing_field("production_type"))
    results.append(check_for_gaps())
    results.append(get_outliers())

    return results

In [7]:
import pandas as pd
from IPython.display import display, Markdown

dq_results = run_data_quality_checks()
dq_report_df = pd.DataFrame(dq_results)
display(Markdown("### Data Quality Report"))
display(dq_report_df)

25/06/29 19:22:33 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


### Data Quality Report

,check,status,failed_records
0,production_type not null,Pass,0
1,Data Gaps,Fail,"failed on days [{'day': datetime.date(2025, 6,..."
2,Data Outliers,Fail,"Have outliers {'Fossil gas': 7, 'Hydro Run-of-..."
